In [31]:
import numpy as np
import pandas as pd

In [7]:
4*.015/(np.pi*.1)

0.1909859317102744

In [8]:
class Winslow(object):
    def __init__(self, rotorradius=.1, bladenumber=4, solidity=.2):
        self.rotorradius = rotorradius
        self.bladenumber = bladenumber
        self.solidity = solidity
        
    def rotormass(self):
        return 0.0195*self.rotorradius**(2.0859)*self.solidity**(-.2038)*self.bladenumber**(.5344)
    
    def batterymass(self, capacity, seriesnumber):
        return .0418*capacity**(.9327)*seriesnumber**(1.0725)
    
    def motormassBL(self, Kv, P, I):
        lBL = 4.8910*I**(.1751)*P**(.2476)
        dBL = 41.45*Kv**(-.1919)*P**(.1935)
        return .0109*Kv**(.5122)*P**(-.1902)*np.log10(lBL)**(2.5582)*np.log10(dBL)**(12.8502)
    
    def motormassDC(self, P, Qmax):
        lDC = 20.83*Qmax**(.1924)
        dDC = 11.13*Qmax**(.2895)
        return (10**(-81))*P**(-.2979)*Qmax**(-20.5615)*np.log10(lDC)**(746)*np.log10(dDC)**(-212)
    
    def ESCmassBL(self, Imax):
        return 0.8013*Imax**(.9727)
    
    def ESCmassDC(self, Imax):
        return .977*Imax**(.8483)
    
    def airframemass(self, R, capacity, seriesnumber):
        return 1.3119*R**(1.2767)*self.batterymass(capacity, seriesnumber)**(.4587)
            
    @property
    def rotorradius(self):
        return self._rotorradius
    
    @rotorradius.setter
    def rotorradius(self, rr):
        try:
            rr = np.abs(float(rr))
        except:
            pass
        else:
            self._rotorradius = rr
            
    @property
    def bladenumber(self):
        return self._bladenumber
    
    @bladenumber.setter
    def bladenumber(self, bn):
        try:
            bn = np.abs(int(bn))
        except:
            pass
        else:
            self._bladenumber = bn
            
    @property
    def solidity(self):
        return self._solidity
    
    @solidity.setter
    def solidity(self, sol):
        try:
            sol = np.abs(float(sol))
        except:
            pass
        else:
            self._rotorradius = sol

Power estimate:
* Raspberry Pi: 5V/0.6-1.25A
* Navio2: 4.75-5.25V/0.15A

Mass estimate:
* Raspberry Pi: 0.045 kg
* Navio2: 0.023 kg

In [105]:
# Hover time [s]
HT = 20*60
# Cells in series
S = 4
# Payload mass
m_PL = 0.5
# Thrust-to-weight ratio
TWR = 2
# Avionics-to-motor power ratio
AMR = .1
# Voltage
V = 11.1


T_prop = (m_PL*9.80665*TWR/4)/np.sqrt(2)
W_prop = interpMotorProps_Thrust(T_prop, voltage=11.1, prop="APC1047")["Watts"]
AMR = 5.1*1.15/W_prop

W_PL = (1+AMR/4)*W_prop*4
Ah = W_PL*(HT/3600)/V

In [106]:
AMR

0.3599879240437465

In [107]:
Ah*1e3

2133.1487300812846

In [110]:
# Required mAh of single cell
Ah*1e3/S

533.2871825203212

In [68]:
X2212V3KV980[11.1]["APC9045"]

,Amps,Thrust,Watts,Efficiency,RPM
0,0.9,0.980665,9.99,10.01,3406.0
1,2.1,1.961330,23.31,8.58,4827.0
2,3.3,2.941995,36.63,8.19,5789.0
3,4.8,3.922660,53.28,7.51,6642.0
4,6.5,4.903325,72.15,6.93,7370.0
5,12.2,7.698220,135.42,5.80,9308.0


In [67]:
T_prop

3.4671743578615275

In [66]:
W_prop

45.54663953378007

In [5]:
uav = Winslow()

In [11]:
uav.bladenumber = -2.3

In [12]:
uav.bladenumber

2

Source: https://sunnyskyusa.com/collections/x-motors/products/sunnsky-x2212

In [59]:
tableAPC8038 = """
0.7	100	10.36	9.65	4120
1.7	200	25.16	7.95	5641
2.8	300	41.44	7.24	6668
4	400	59.2	6.76	7629
5.4	500	79.92	6.26	8440
7.1	600	105.08	5.71	9154
8.7	700	128.76	5.44	9766
10.2	800	150.96	5.30	10436
16.4	1075	242.72	4.43	11910
"""

tableAPC9045_11 = """
0.9	100	9.99	10.01	3406
2.1	200	23.31	8.58	4827
3.3	300	36.63	8.19	5789
4.8	400	53.28	7.51	6642
6.5	500	72.15	6.93	7370
12.2	785	135.42	5.80	9308
"""

tableAPC9045_15 = """
0.6	100	8.88	11.26	3452
1.5	200	22.2	9.01	4734
2.7	300	39.96	7.51	5780
3.8	400	56.24	7.11	667
5.2	500	76.96	6.50	746
6.6	600	97.68	6.14	8140
8.4	700	124.32	5.63	8649
10	800	148	5.41	9150
11.9	900	176.12	5.11	9696
14	1000	207.2	4.83	11368
16	1100	236.8	4.65	10221
18.2	1220	269.36	4.53	10649
"""

tableAPC9047_11 = """
0.8	100	8.88	11.26	3288
1.9	200	21.09	9.48	4580
3.2	300	35.52	8.45	5638
4.9	400	54.39	7.35	6591
6.4	500	71.04	7.04	7301
8.2	600	91.02	6.59	8041
10.4	700	115.44	6.06	9396
13.2	840	146.52	5.73	8730
"""

tableAPC9047_15 = """
0.6	100	8.88	11.26	3231
1.5	200	22.2	9.01	4643
2.5	300	37	8.11	5605
3.8	400	56.24	7.11	6432
5	500	74	6.76	7378
6.5	600	96.2	6.24	8090
8.2	700	121.36	5.77	8730
10	800	148	5.41	9366
11.7	900	173.16	5.20	10131
13.8	1000	204.24	4.90	10716
18.4	1250	272.32	4.59	11755
"""

tableAPC1038 = """
0.7	100	7.77	12.87	2813
1.8	200	19.98	10.01	3866
3	300	33.3	9.01	4636
4.6	400	51.06	7.83	5170
6.3	500	69.93	7.15	5687
8.1	600	89.91	6.67	6100
10.2	700	113.22	6.18	6591
12.4	800	137.64	5.81	6853
15.3	900	169.83	5.30	7262
20.2	1130	224.22	5.04	7923
"""

tableAPC1047_11 = """
0.7	100	7.77	12.87	2612
1.7	200	18.87	10.60	3605
3	300	33.3	9.01	4264
4.5	400	49.95	8.01	4887
6.4	500	71.04	7.04	5418
8.3	600	92.13	6.51	5872
10.45	700	115.99	6.03	6335
12.7	800	5	5.67	6983
15.3	900	140.97	5.30	7378
19.9	1100	169.83	4.98	7748
"""

tableAPC1047_15 = """
0.5	100	220.89	13.51	2600
1.3	200	7.4	10.40	3593
2.2	300	19.24	9.21	4253
3.4	400	32.56	7.95	4869
4.9	500	50.32	6.89	5381
6.5	600	72.52	6.24	5845
8.7	700	96.2	5.44	6332
10.5	800	128.76	5.15	6610
12.4	900	155.4	4.90	7055
14.7	1000	183.52	4.60	7444
16.4	1100	217.56	4.53	7766
18	1200	242.72	4.50	8100
20.6	1300	266.4	4.26	8394
24.5	1400	304.88	3.86	8620
27.4	1500	362.6	3.70	8900
29	1650	405.52	3.84	9409
"""

def table2DF(table):
    table = np.asarray([[float(subval) for subval in val.split("\t")] for val in table.split('\n') if val != ""])
    df = pd.DataFrame({
        "Amps" : table[:,0],
        "Thrust" : table[:,1]*9.80665*1e-3,
        "Watts" : table[:,2],
        "Efficiency" : table[:,3],
        "RPM" : table[:,4],
    })
    
    return df

tableAPC8038 = table2DF(tableAPC8038)
tableAPC9045_11 = table2DF(tableAPC9045_11)
tableAPC9045_15 = table2DF(tableAPC9045_15)
tableAPC9047_11 = table2DF(tableAPC9047_11)
tableAPC9047_15 = table2DF(tableAPC9047_15)
tableAPC1038 = table2DF(tableAPC1038)
tableAPC1047_11 = table2DF(tableAPC1047_11)
tableAPC1047_15 = table2DF(tableAPC1047_15)

In [76]:
X2212V3KV980 = {
    11.1: {
        "APC9045" : tableAPC9045_11,
        "APC9047" : tableAPC9047_11,
        "APC1038" : tableAPC1038,
        "APC1047" : tableAPC1047_11
    },
    14.8: {
        "APC8038" : tableAPC8038,
        "APC9045" : tableAPC9045_15,
        "APC9047" : tableAPC9047_15,
        "APC1047" : tableAPC1047_15
    }
}

engineDB = {
    "X2212V3_KV980" : X2212V3KV980
}

In [52]:
def interpMotorProps_Thrust(T, voltage=11.1, model="X2212V3_KV980", prop="APC9045"):
    A = np.interp([T],engineDB[model][voltage][prop]["Thrust"],engineDB[model][voltage][prop]["Amps"])[0]
    RPM = np.interp([T],engineDB[model][voltage][prop]["Thrust"],engineDB[model][voltage][prop]["RPM"])[0]
    W = np.interp([T],engineDB[model][voltage][prop]["Thrust"],engineDB[model][voltage][prop]["Watts"])[0]
    
    return dict(Amps=A, RPM=RPM, Watts=W)

In [54]:
interpMotorProps_Thrust(3.5)["Watts"]

46.10396231128877

In [49]:
engineDB = {
    "X2212V3_KV980" : X2212V3KV980
}

In [61]:
X2212V3KV980[11.1]["APC9045"]

,Amps,Thrust,Watts,Efficiency,RPM
0,0.9,0.980665,9.99,10.01,3406.0
1,2.1,1.961330,23.31,8.58,4827.0
2,3.3,2.941995,36.63,8.19,5789.0
3,4.8,3.922660,53.28,7.51,6642.0
4,6.5,4.903325,72.15,6.93,7370.0
5,12.2,7.698220,135.42,5.80,9308.0
